In [ ]:
#LOAD FUNCTIONS

In [ ]:
%run /CommonNotificationHeader 

# ReadParquetDay Function

In [ ]:
#Config
#Need create scope databricks to KayVault
#dbutils.secrets.get(scope="Databricks-Scope",key="Datalake-KEY")

spark.conf.set("fs.azure.account.key." + accountDatalake + ".dfs.core.windows.net", keyDatalake)
spark.conf.set("fs.azure.account.key." + accountDatalake + ".blob.core.windows.net", keyDatalake)



def readParquetDay(pathTable,year,month,day,hour):
    try:
        df_tab = sqlContext.read.format("parquet").option("wholeFile", "true").option("header", "true").option("inferSchema", "true").load(datalakeURL+pathTable+"/"+str(year)+"/"+str(month)+"/"+str(day)+"/"+str(hour)+"/*")
        print(pathTable)
        print("Ficheiro lido: {0}/{1}/{2}/{3}/{4}".format(pathTable, year, month, day,hour))
        return df_tab
    except Exception as e:
        print(e)
        print("Ficheiro inexistente: {0}/{1}/{2}/{3}/{4}".format(pathTable, year, month, day,hour))

In [ ]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,LongType,DateType
from pyspark.sql.functions import from_json

#Config EventHub
connectionString = "Endpoint=sb://xxxxx.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=xxxx;EntityPath=xxxxx"

ehConf = {
  'eventhubs.connectionString' : connectionString,
  'eventhubs.consumerGroup': 'data-parcel'
}

ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)

#Schema for Read Event
Schema = StructType([
    StructField('providerTerminalId', StringType(), False),
    StructField('providerParcelId', StringType(), True),
    StructField('partnerParcelLabel', StringType(), True),
    StructField('doors', StringType(), True),
    StructField('parcelStatusEnum', StringType(), True),
    StructField('eventDate', StringType(), True),
    StructField('parcel', StructType([
        StructField('id', StringType(), True),
        StructField('status', StringType(), True),
        StructField('courierCompany', StringType(), True),
        StructField('originalCourierLabel', StringType(), True),
        StructField('parcelPartnerLabel', StringType(), True),
        StructField('partner', StringType(), True),
        StructField('reservationDate', StringType(), True),
        StructField('expirationDate', StringType(), True),
        StructField('depositedDate', StringType(), True),
        StructField('collectedDate', StringType(), True),
        StructField('returnedDate', StringType(), True),
        StructField('createdDate', StringType(), True),
        StructField('lastModifiedDate', StringType(), True),
        StructField('customerEmail', StringType(), True),
        StructField('customerPhone', StringType(), True),
        StructField('customerName', StringType(), True),
        StructField('parcelHeight', StringType(), True),
        StructField('parcelWidth', StringType(), True),
        StructField('parcelDepth', StringType(), True),
        StructField('parcelWeight', StringType(), True),
        StructField('payed', StringType(), True),
        StructField('climatizationType',StringType(),True),
        StructField('epv', StringType(), True),
        StructField('depositOffsetLimit', StringType(), True),
        StructField('collectOffsetLimit', StringType(), True), 
        StructField('terminalExternalId', StringType(), True),
        StructField('eventDate', StringType(), True),
        StructField('doors', StringType(), True),
        StructField('providerTerminalId', StringType(), True)        
    ]))
])



In [ ]:
#Read data from eventHub
df = spark \
  .readStream  \
  .format("eventhubs") \
  .option("maxEventsPerTrigger",5000) \
  .option("startingOffsets", "latest") \
  .options(**ehConf) \
  .load()

#Mapping data to Json
from pyspark.sql.functions import from_json
df = df\
    .selectExpr("cast(body as string) as json")\
    .select(from_json("json", Schema).alias("data"))\
    .select("data.*")\

#Mapping columns
df_columns = df.select("*")\
        .withColumn("providerTerminalId",df.providerTerminalId)\
        .withColumn("providerParcelId",df.providerParcelId)\
        .withColumn("partnerParcelLabel",df.partnerParcelLabel)\
        .withColumn("doors",df.doors)\
        .withColumn("parcelStatusEnum",df.parcelStatusEnum)\
        .withColumn("eventDate",df.eventDate)\
        .withColumn("id",df.parcel.id)\
        .withColumn("status",df.parcel.status)\
        .withColumn("courierCompany",df.parcel.courierCompany)\
        .withColumn("originalCourierLabel",df.parcel.originalCourierLabel)\
        .withColumn("Parcel_parcelPartnerLabel",df.parcel.parcelPartnerLabel)\
        .withColumn("partner",df.parcel.partner)\
        .withColumn("reservationDate",df.parcel.reservationDate)\
        .withColumn("expirationDate",df.parcel.expirationDate)\
        .withColumn("depositedDate",df.parcel.depositedDate)\
        .withColumn("collectedDate",df.parcel.collectedDate)\
        .withColumn("returnedDate",df.parcel.returnedDate)\
        .withColumn("createdDate",df.parcel.createdDate)\
        .withColumn("lastModifiedDate",df.parcel.lastModifiedDate)\
        .withColumn("customerEmail",df.parcel.customerEmail)\
        .withColumn("customerPhone",df.parcel.customerPhone)\
        .withColumn("customerName",df.parcel.customerName)\
        .withColumn("parcelHeight",df.parcel.parcelHeight)\
        .withColumn("parcelWidth",df.parcel.parcelWidth)\
        .withColumn("parcelDepth",df.parcel.parcelDepth)\
        .withColumn("parcelWeight",df.parcel.parcelWeight)\
        .withColumn("payed",df.parcel.payed)\
        .withColumn("climatizationType",df.parcel.climatizationType)\
        .withColumn("epv",df.parcel.epv)\
        .withColumn("depositOffsetLimit",df.parcel.depositOffsetLimit)\
        .withColumn("collectOffsetLimit",df.parcel.collectOffsetLimit)\
        .withColumn("terminalExternalId",df.parcel.terminalExternalId)\
        .withColumn("Parcel_eventDate",df.parcel.eventDate)\
        .withColumn("Parcel_doors",df.parcel.doors)\
        .withColumn("Parcel_providerTerminalId",df.parcel.providerTerminalId)\


#Write in delta format events
ds = df_columns \
  .writeStream \
  .format("delta") \
  .trigger(once=True) \
  .outputMode("append") \
  .options(**ehConf) \
  .option("checkpointLocation", datalakeURL + "RAW/DELTA/PATH" ) \
  .start(datalakeURL + "RAW/DELTA/PATH")

## Validation of Data Load in DataLake

In [ ]:
def readDataDelta(pathTable):
    try:
        df_tab = sqlContext.read.format("delta").option("wholeFile", "true").option("header", "true").option("inferSchema", "true").load(datalakeURL+pathTable)
        print(pathTable)
        return df_tab
    except Exception as e:
        print(e)
        
returndf = readDataDelta("RAW/DELTA/PATH")

returndf.createOrReplaceTempView('')